In [ ]:
"""
\********************************************************************************
* Copyright (c) 2024 the Qrisp authors
*
* This program and the accompanying materials are made available under the
* terms of the Eclipse Public License 2.0 which is available at
* http://www.eclipse.org/legal/epl-2.0.
*
* This Source Code may also be made available under the following Secondary
* Licenses when the conditions for such availability set forth in the Eclipse
* Public License, v. 2.0 are satisfied: GNU General Public License, version 2
* with the GNU Classpath Exception which is
* available at https://www.gnu.org/software/classpath/license.html.
*
* SPDX-License-Identifier: EPL-2.0 OR GPL-2.0 WITH Classpath-exception-2.0
********************************************************************************/
"""

from state_preparation_parametrized import *
from qrisp.quantum_backtracking import OHQInt
from classical_cost_func import cost_function, format_coeffs, cost_symp
import numpy as np
import matplotlib.pyplot as plt
from qrisp.qaoa import *
from encode_PBS_problem import PBS_graph, N, M, cost_coeff


tot_coeff = format_coeffs(cost_coeff, N )

qtype = OHQInt(N)
q_array = QuantumArray(qtype = qtype, shape = (M))

# Cost function as SymPy polynomial
C,S = cost_symp(tot_coeff,M,N,PBS_graph)
ord_symbs=list(S.values())

def cost_op(q_array, gamma):
    app_sb_phase_polynomial(q_array, C, ord_symbs, t=-gamma)

# Initialization function
init_func = pbs_state_init(PBS_graph, 0 , N)

# Inverse state preparation operator
def inv_init_func(q_array):
    with invert():
        init_func(q_array)

# Define mixer operator
def mixer_op(q_array, beta):
    with conjugate(inv_init_func)(q_array):
        for i in range(len(q_array)):
            mcp(beta, q_array[i], ctrl_state = 0)

# Classical cost function
values = []
cl_cost = cost_function(tot_coeff,PBS_graph,values)

# Define QAOA problem
qaoaPBS = QAOAProblem(cost_operator=cost_op ,mixer=mixer_op, cl_cost_function=cl_cost)
qaoaPBS.set_init_function(init_func)
depth = 2#int(np.log2(M*N))

In [ ]:
circ,symbols=qaoaPBS.compile_circuit(q_array,depth)

In [ ]:
def evaluate_cost(theta,symbols):
    subs_dic = {symbols[i] : theta[i] for i in range(len(symbols))}
    res_dic = q_array.get_measurement(subs_dic = subs_dic, precompiled_qc = circ)
    cost = cl_cost(res_dic)
    return cost

In [ ]:
def finite_diff(theta,index,gb,eps=.2):
    e_i = np.identity(theta.size)[:, index]
    plus = theta + eps * e_i
    minus = theta - eps * e_i
    grad = (evaluate_cost(plus,gb) - evaluate_cost(minus,gb)) / (2 * eps)
    return grad

In [ ]:

def compute_gradients(gb, grad_func, num_samples=10):
    grads = []
    index = len(gb)-1 #wrt last parameter

    for _ in range(num_samples): #sampling from random samples
        theta = 2*np.pi* np.random.rand(len(gb)) 
        grad = grad_func(theta,index,gb)
        grads.append(grad)
    grads = np.array(grads)
    return grads

In [ ]:
gradients=compute_gradients(symbols,finite_diff,100)

In [ ]:
np.var(gradients)